<a href="https://colab.research.google.com/github/mengwangk/dl-projects/blob/master/02_2_feature_engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

## Featuretools

In [0]:
import featuretools as ft
import pandas as pd
import numpy as np
from featuretools.entityset import EntitySet, Entity

In [17]:
 data = ft.demo.load_mock_customer()
 type(data), data.keys

(dict, <function dict.keys>)

In [18]:
customers_df = data["customers"]
type(customers_df), customers_df

(pandas.core.frame.DataFrame,
    customer_id zip_code           join_date date_of_birth
 0            1    60091 2011-04-17 10:48:33    1994-07-18
 1            2    13244 2012-04-15 23:31:04    1986-08-18
 2            3    13244 2011-08-13 15:42:34    2003-11-21
 3            4    60091 2011-04-08 20:08:14    2006-08-15
 4            5    60091 2010-07-17 05:27:50    1984-07-28)

In [20]:
sessions_df = data["sessions"]
sessions_df.sample(5)

,session_id,customer_id,device,session_start
13,14,1,tablet,2014-01-01 03:28:00
6,7,3,tablet,2014-01-01 01:39:40
1,2,5,mobile,2014-01-01 00:17:20
28,29,1,mobile,2014-01-01 07:10:05
24,25,3,desktop,2014-01-01 05:59:40


In [21]:
transactions_df = data["transactions"]
transactions_df.sample(5)

,transaction_id,session_id,transaction_time,product_id,amount
74,232,5,2014-01-01 01:20:10,1,139.20
231,27,17,2014-01-01 04:10:15,2,90.79
434,36,31,2014-01-01 07:50:10,3,62.35
420,56,30,2014-01-01 07:35:00,3,72.70
54,444,4,2014-01-01 00:58:30,4,43.59


In [0]:
entities = {
    "customers" : (customers_df, "customer_id"),
     "sessions" : (sessions_df, "session_id", "session_start"),
     "transactions" : (transactions_df, "transaction_id", "transaction_time")
}

In [0]:
relationships = [("sessions", "session_id", "transactions", "session_id"),
                  ("customers", "customer_id", "sessions", "customer_id")]

In [0]:
feature_matrix_customers, features_defs = ft.dfs(entities=entities,
                                          relationships=relationships,
                                          target_entity="customers")

In [27]:
feature_matrix_customers

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.product_id),MODE(transactions.product_id),DAY(join_date),DAY(date_of_birth),YEAR(join_date),YEAR(date_of_birth),MONTH(join_date),MONTH(date_of_birth),WEEKDAY(join_date),WEEKDAY(date_of_birth),SUM(sessions.STD(transactions.amount)),SUM(sessions.MAX(transactions.amount)),SUM(sessions.SKEW(transactions.amount)),SUM(sessions.MIN(transactions.amount)),SUM(sessions.MEAN(transactions.amount)),SUM(sessions.NUM_UNIQUE(transactions.product_id)),STD(sessions.SUM(transactions.amount)),STD(sessions.MAX(transactions.amount)),STD(sessions.SKEW(transactions.amount)),STD(sessions.MIN(transactions.amount)),STD(sessions.MEAN(transactions.amount)),STD(sessions.COUNT(transactions)),STD(sessions.NUM_UNIQUE(transactions.product_id)),MAX(sessions.SUM(transactions.amount)),MAX(sessions.STD(transactions.amount)),MAX(sessions.SKEW(transactions.amount)),MAX(sessions.MIN(transactions.amount)),MAX(sessions.MEAN(transactions.amount)),MAX(sessions.COUNT(transactions)),MAX(sessions.NUM_UNIQUE(transactions.product_id)),SKEW(sessions.SUM(transactions.amount)),SKEW(sessions.STD(transactions.amount)),SKEW(sessions.MAX(transactions.amount)),SKEW(sessions.MIN(transactions.amount)),SKEW(sessions.MEAN(transactions.amount)),SKEW(sessions.COUNT(transactions)),SKEW(sessions.NUM_UNIQUE(transactions.product_id)),MIN(sessions.SUM(transactions.amount)),MIN(sessions.STD(transactions.amount)),MIN(sessions.MAX(transactions.amount)),MIN(sessions.SKEW(transactions.amount)),MIN(sessions.MEAN(transactions.amount)),MIN(sessions.COUNT(transactions)),MIN(sessions.NUM_UNIQUE(transactions.product_id)),MEAN(sessions.SUM(transactions.amount)),MEAN(sessions.STD(transactions.amount)),MEAN(sessions.MAX(transactions.amount)),MEAN(sessions.SKEW(transactions.amount)),MEAN(sessions.MIN(transactions.amount)),MEAN(sessions.MEAN(transactions.amount)),MEAN(sessions.COUNT(transactions)),MEAN(sessions.NUM_UNIQUE(transactions.product_id)),NUM_UNIQUE(sessions.MODE(transactions.product_id)),NUM_UNIQUE(sessions.DAY(session_start)),NUM_UNIQUE(sessions.YEAR(session_start)),NUM_UNIQUE(sessions.MONTH(session_start)),NUM_UNIQUE(sessions.WEEKDAY(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.DAY(session_start)),MODE(sessions.YEAR(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.WEEKDAY(session_start))
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60091,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,126,5,4,17,18,2011,1994,4,7,6,0,312.745952,1057.97,-0.476122,78.59,582.193117,40,279.510713,7.322191,0.589386,6.954507,13.759314,4.062019,0.000000,1613.93,46.905665,0.640252,26.36,88.755625,25,5,0.778170,-0.312355,-0.780493,2.440005,-0.424949,1.946018,0.000000,809.97,30.450261,118.90,-1.038434,50.623125,12,5,1128.202500,39.093244,132.246250,-0.059515,9.823750,72.774140,15.750000,5.000000,4,1,1,1,1,4,1,2014,1,2
2,13244,7,3,desktop,7200.28,37.705178,146.81,0.098259,8.73,77.422366,93,5,4,15,18,2012,1986,4,8,6,0,258.700528,931.63,-0.277640,154.60,548.905851,35,251.609234,17.221593,0.509798,15.874374,11.477071,3.450328,0.000000,1320.64,47.935920,0.755711,56.46,96.581000,18,5,-0.440929,0.013087,-1.539467,2.154929,0.235296,-0.303276,0.000000,634.84,27.839228,100.04,-0.763603,61.910000,8,5,1028.611429,36.957218,133.090000,-0.039663,22.085714,78.415122,13.285714,5.000000,4,1,1,1,1,3,1,2014,1,2
3,13244,6,3,desktop,6236.62,43.683296,149.15,0.418230,5.89,67.060430,93,5,1,13,21,2011,2003,8,11,5,4,257.299895,847.63,2.286086,66.21,405.237462,29,219.021420,10.724241,0.429374,5.424407,11.174282,2.428992,0.408248,1477.97,50.110120,0.854976,20.06,82.109444,18,5,2.246479,-0.245703,-0.941078,1.000771,0.678544,-1.507217,-2.449490,889.21,35.704680,126.74,-0.289466,55.579412,

In [29]:
features_defs

[<Feature: zip_code>,
 <Feature: COUNT(sessions)>,
 <Feature: NUM_UNIQUE(sessions.device)>,
 <Feature: MODE(sessions.device)>,
 <Feature: SUM(transactions.amount)>,
 <Feature: STD(transactions.amount)>,
 <Feature: MAX(transactions.amount)>,
 <Feature: SKEW(transactions.amount)>,
 <Feature: MIN(transactions.amount)>,
 <Feature: MEAN(transactions.amount)>,
 <Feature: COUNT(transactions)>,
 <Feature: NUM_UNIQUE(transactions.product_id)>,
 <Feature: MODE(transactions.product_id)>,
 <Feature: DAY(join_date)>,
 <Feature: DAY(date_of_birth)>,
 <Feature: YEAR(join_date)>,
 <Feature: YEAR(date_of_birth)>,
 <Feature: MONTH(join_date)>,
 <Feature: MONTH(date_of_birth)>,
 <Feature: WEEKDAY(join_date)>,
 <Feature: WEEKDAY(date_of_birth)>,
 <Feature: SUM(sessions.STD(transactions.amount))>,
 <Feature: SUM(sessions.MAX(transactions.amount))>,
 <Feature: SUM(sessions.SKEW(transactions.amount))>,
 <Feature: SUM(sessions.MIN(transactions.amount))>,
 <Feature: SUM(sessions.MEAN(transactions.amount))>,
 <

In [0]:
feature_matrix_sessions, features_defs = ft.dfs(entities=entities,
                                                  relationships=relationships,
                                                  target_entity="sessions")



In [31]:
feature_matrix_sessions

,customer_id,device,SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.product_id),MODE(transactions.product_id),DAY(session_start),YEAR(session_start),MONTH(session_start),WEEKDAY(session_start),customers.zip_code,NUM_UNIQUE(transactions.DAY(transaction_time)),NUM_UNIQUE(transactions.YEAR(transaction_time)),NUM_UNIQUE(transactions.MONTH(transaction_time)),NUM_UNIQUE(transactions.WEEKDAY(transaction_time)),MODE(transactions.DAY(transaction_time)),MODE(transactions.YEAR(transaction_time)),MODE(transactions.MONTH(transaction_time)),MODE(transactions.WEEKDAY(transaction_time)),customers.COUNT(sessions),customers.NUM_UNIQUE(sessions.device),customers.MODE(sessions.device),customers.SUM(transactions.amount),customers.STD(transactions.amount),customers.MAX(transactions.amount),customers.SKEW(transactions.amount),customers.MIN(transactions.amount),customers.MEAN(transactions.amount),customers.COUNT(transactions),customers.NUM_UNIQUE(transactions.product_id),customers.MODE(transactions.product_id),customers.DAY(join_date),customers.DAY(date_of_birth),customers.YEAR(join_date),customers.YEAR(date_of_birth),customers.MONTH(join_date),customers.MONTH(date_of_birth),customers.WEEKDAY(join_date),customers.WEEKDAY(date_of_birth)
session_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,desktop,1229.01,41.600976,141.66,0.295458,20.91,76.813125,16,5,3,1,2014,1,2,13244,1,1,1,1,1,2014,1,2,7,3,desktop,7200.28,37.705178,146.81,0.098259,8.73,77.422366,93,5,4,15,18,2012,1986,4,8,6,0
2,5,mobile,746.96,45.893591,135.25,-0.160550,9.32,74.696000,10,5,5,1,2014,1,2,60091,1,1,1,1,1,2014,1,2,6,3,mobile,6349.66,44.095630,149.02,-0.025941,7.55,80.375443,79,5,5,17,28,2010,1984,7,7,5,5
3,4,mobile,1329.00,46.240016,147.73,-0.324012,8.70,88.600000,15,5,1,1,2014,1,2,60091,1,1,1,1,1,2014,1,2,8,3,mobile,8727.68,45.068765,149.95,-0.036348,5.73,80.070459,109,5,2,8,15,2011,2006,4,8,4,1
4,1,mobile,1613.93,40.187205,129.00,0.234349,6.29,64.557200,25,5,5,1,2014,1,2,60091,1,1,1,1,1,2014,1,2,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,126,5,4,17,18,2011,1994,4,7,6,0
5,4,mobile,777.02,48.918663,139.20,0.336381,7.43,70.638182,11,5,5,1,2014,1,2,60091,1,1,1,1,1,2014,1,2,8,3,mobile,8727.68,45.068765,149.95,-0.036348,5.73,80.070459,109,5,2,8,15,2011,2006,4,8,4,1
6,1,tablet,1266.60,42.654755,139.23,-0.134754,8.74,84.440000,15,5,4,1,2014,1,2,60091,1,1,1,1,1,2014,1,2,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,126,5,4,17,18,2011,1994,4,7,6,0
7,3,tablet,941.87,47.264797,146.31,0.618455,8.19,62.791333,15,5,1,1,2014,1,2,13244,1,1,1,1,1,2014,1,2,6,3,desktop,6236.62,43.683296,149.15,0.418230,5.89,67.060430,93,5,1,13,21,2011,2003,8,11,5,4
8,4,tablet,1351.46,44.213242,143.85,0.200676,12.59,75.081111,18,5,1,1,2014,1,2,60091,1,1,1,1,1,2014,1,2,8,3,mobile,8727.68,45.068765,149.95,-0.036348,5.73,80.070459,109,5,2,8,15,2011,2006,4,8,4,1
9,1,desktop,1052.03,46.905665,132.72,0.226337,5.81,70.135333,15,5,1,1,2014,1,2,60091,1,1,1,1,1,2014,1,2,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,126,5,4,17,18,2011,1994,4,7,6,0


### EntitySets

In [0]:
merged_df =  data["transactions"].merge(data["sessions"]).merge(data["customers"])

In [33]:
merged_df.sample(5)

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code,join_date,date_of_birth
372,479,26,2014-01-01 06:30:00,2,57.89,1,tablet,2014-01-01 06:17:00,60091,2011-04-17 10:48:33,1994-07-18
467,114,34,2014-01-01 08:25:55,4,25.73,3,desktop,2014-01-01 08:24:50,13244,2011-08-13 15:42:34,2003-11-21
241,81,13,2014-01-01 03:15:00,2,5.73,4,mobile,2014-01-01 03:15:00,60091,2011-04-08 20:08:14,2006-08-15
268,87,21,2014-01-01 05:18:30,5,7.93,4,desktop,2014-01-01 05:02:15,60091,2011-04-08 20:08:14,2006-08-15
139,107,28,2014-01-01 06:58:10,1,32.92,5,mobile,2014-01-01 06:50:35,60091,2010-07-17 05:27:50,1984-07-28


In [34]:
merged_df.columns

Index(['transaction_id', 'session_id', 'transaction_time', 'product_id',
       'amount', 'customer_id', 'device', 'session_start', 'zip_code',
       'join_date', 'date_of_birth'],
      dtype='object')

In [35]:
data["customers"].columns

Index(['customer_id', 'zip_code', 'join_date', 'date_of_birth'], dtype='object')

In [0]:
products_df = data["products"]

In [37]:
products_df

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


#### Create EntitySet

In [0]:
es = ft.EntitySet(id="customer_data")

In [0]:
es = es.entity_from_dataframe(entity_id="transactions",
                                  dataframe=merged_df,
                                  index="transaction_id",
                                  time_index="transaction_time",
                                  variable_types={"product_id": ft.variable_types.Categorical,
                                                  "zip_code": ft.variable_types.Ordinal})

In [44]:
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
  Relationships:
    No relationships

In [53]:
es["transactions"].variables

[<Variable: transaction_id (dtype = index)>,
 <Variable: session_id (dtype = numeric)>,
 <Variable: transaction_time (dtype: datetime_time_index, format: None)>,
 <Variable: amount (dtype = numeric)>,
 <Variable: customer_id (dtype = numeric)>,
 <Variable: device (dtype = categorical)>,
 <Variable: session_start (dtype: datetime, format: None)>,
 <Variable: join_date (dtype: datetime, format: None)>,
 <Variable: date_of_birth (dtype: datetime, format: None)>,
 <Variable: product_id (dtype = categorical)>,
 <Variable: zip_code (dtype = ordinal)>]

In [54]:
 es = es.entity_from_dataframe(entity_id="products",
                                dataframe=products_df,
                                  index="product_id")
 es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    No relationships

In [0]:
 new_relationship = ft.Relationship(es["products"]["product_id"],
                                      es["transactions"]["product_id"])

In [56]:
es.add_relationship(new_relationship)

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    transactions.product_id -> products.product_id

In [0]:
# Create new entity from existing table
es = es.normalize_entity(base_entity_id="transactions",
                         new_entity_id="sessions",
                          index="session_id",
                          make_time_index="session_start",
                          additional_variables=["device", "customer_id", "zip_code", "session_start", "join_date"])
    

In [59]:
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 6]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id

In [60]:
es["transactions"].variables

[<Variable: transaction_id (dtype = index)>,
 <Variable: session_id (dtype = id)>,
 <Variable: transaction_time (dtype: datetime_time_index, format: None)>,
 <Variable: amount (dtype = numeric)>,
 <Variable: date_of_birth (dtype: datetime, format: None)>,
 <Variable: product_id (dtype = categorical)>]

In [61]:
es["sessions"].variables

[<Variable: session_id (dtype = index)>,
 <Variable: device (dtype = categorical)>,
 <Variable: customer_id (dtype = numeric)>,
 <Variable: zip_code (dtype = ordinal)>,
 <Variable: session_start (dtype: datetime_time_index, format: None)>,
 <Variable: join_date (dtype: datetime, format: None)>]

In [62]:
es["sessions"].df.head(5)

,session_id,device,customer_id,zip_code,session_start,join_date
1,1,desktop,2,13244,2014-01-01 00:00:00,2012-04-15 23:31:04
2,2,mobile,5,60091,2014-01-01 00:17:20,2010-07-17 05:27:50
3,3,mobile,4,60091,2014-01-01 00:28:10,2011-04-08 20:08:14
4,4,mobile,1,60091,2014-01-01 00:44:25,2011-04-17 10:48:33
5,5,mobile,4,60091,2014-01-01 01:11:30,2011-04-08 20:08:14


In [63]:
es["transactions"].df.head(5)

,transaction_id,session_id,transaction_time,amount,date_of_birth,product_id
298,298,1,2014-01-01 00:00:00,127.64,1986-08-18,5
2,2,1,2014-01-01 00:01:05,109.48,1986-08-18,2
308,308,1,2014-01-01 00:02:10,95.06,1986-08-18,3
116,116,1,2014-01-01 00:03:15,78.92,1986-08-18,4
371,371,1,2014-01-01 00:04:20,31.54,1986-08-18,3


In [0]:
es = es.normalize_entity(base_entity_id="sessions",
                             new_entity_id="customers",
                             index="customer_id",
                             make_time_index="join_date",
                             additional_variables=["zip_code", "join_date"])

In [71]:
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 4]
    customers [Rows: 5, Columns: 3]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id

In [0]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_entity="products")

In [74]:
feature_matrix

,brand,SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.session_id),MODE(transactions.session_id),NUM_UNIQUE(transactions.DAY(transaction_time)),NUM_UNIQUE(transactions.DAY(date_of_birth)),NUM_UNIQUE(transactions.YEAR(transaction_time)),NUM_UNIQUE(transactions.YEAR(date_of_birth)),NUM_UNIQUE(transactions.MONTH(transaction_time)),NUM_UNIQUE(transactions.MONTH(date_of_birth)),NUM_UNIQUE(transactions.WEEKDAY(transaction_time)),NUM_UNIQUE(transactions.WEEKDAY(date_of_birth)),NUM_UNIQUE(transactions.sessions.device),NUM_UNIQUE(transactions.sessions.customer_id),MODE(transactions.DAY(transaction_time)),MODE(transactions.DAY(date_of_birth)),MODE(transactions.YEAR(transaction_time)),MODE(transactions.YEAR(date_of_birth)),MODE(transactions.MONTH(transaction_time)),MODE(transactions.MONTH(date_of_birth)),MODE(transactions.WEEKDAY(transaction_time)),MODE(transactions.WEEKDAY(date_of_birth)),MODE(transactions.sessions.device),MODE(transactions.sessions.customer_id)
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,B,7489.79,42.479989,149.56,0.125525,6.84,73.429314,102,34,3,1,4,1,5,1,3,1,4,3,5,1,18,2014,1994,1,7,2,0,desktop,1
2,B,7021.43,46.336308,149.95,0.151934,5.73,76.319891,92,34,28,1,4,1,5,1,3,1,4,3,5,1,18,2014,2006,1,8,2,0,desktop,4
3,B,7008.12,38.871405,148.31,0.223938,5.89,73.001250,96,35,1,1,4,1,5,1,3,1,4,3,5,1,18,2014,2006,1,8,2,0,desktop,4
4,B,8088.97,42.492501,146.46,-0.132077,5.81,76.311038,106,34,29,1,4,1,5,1,3,1,4,3,5,1,18,2014,1994,1,7,2,0,desktop,1
5,A,7931.55,42.131902,149.02,0.098248,5.91,76.264904,104,34,4,1,4,1,5,1,3,1,4,3,5,1,18,2014,1994,1,7,2,0,mobile,1
